In [1]:
import json
import pandas as pd
import requests
import matplotlib.pyplot as plt
import scipy as sp
from pprint import pprint
import quandl
from config import quandl_key
import numpy as np

In [3]:
master_csv = pd.read_csv("master_data_cleaned.csv")
master_csv

,Year,Month,Year-Month,Date Zillow,Date Interest,Indicator ID,Indicator Name,Region ID,Region Type,Region Name,Value,County Population,Unemployment Rate,Wells Fargo Rate,Freddie Mac Rate
0,2016,Apr,2016-4,4/30/2016,4/28/2016,Z2BR,ZHVI 2-Bedroom Time Series ($),753871,metro,"Breckenridge, CO; CO",384884.00,30566,3.5,3.88,3.66
1,2016,Apr,2016-4,4/30/2016,4/28/2016,Z4BR,ZHVI 4-Bedroom Time Series ($),753871,metro,"Breckenridge, CO; CO",881951.00,30566,3.5,3.88,3.66
2,2016,Apr,2016-4,4/30/2016,4/28/2016,Z5BR,ZHVI 5+ Bedroom Time Series ($),753871,metro,"Breckenridge, CO; CO",1287769.00,30566,3.5,3.88,3.66
3,2016,Apr,2016-4,4/30/2016,4/28/2016,ZATT,ZHVI All Homes- Top Tier Time Series ($),753871,metro,"Breckenridge, CO; CO",842653.00,30566,3.5,3.88,3.66
4,2016,Apr,2016-4,4/30/2016,4/28/2016,Z1BR,ZHVI 1-Bedroom Time Series ($),753871,metro,"Breckenridge, CO; CO",243688.00,30566,3.5,3.88,3.66
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67019,2020,Sep,2020-9,9/26/2020,NaN,CRAW,"Share of Listings With a Price Cut (Raw, All H...",394645,metro,"Greeley, CO",0.05,332367,6.4,NaN,NaN
67020,2020,Sep,2020-9,9/19/2020,NaN,CRSW,"Share of Listings With a Price Cut (Raw, SFR O...",394645,metro,"Greeley, CO",0.04,332367,6.4,NaN,NaN
67021,2020,Sep,2020-9,9/26/2020,NaN,CRSW,"Share of Listings With a Price Cut (Raw, SFR O...",394645,metro,"Greeley, CO",0.05,332367,6.4,NaN,NaN
67022,2020,Sep,2020-9,9/30/2020,NaN,CRSM,"Share of Listings With a Price Cut (Raw, SFR O...",394645,metro,"Greeley, CO",0.13,332367,6.4,NaN,NaN


In [29]:
#sort master into 4 based on region name
denver_df = master_csv.loc[master_csv['Region Name']=='Denver, CO']
greeley_df = master_csv.loc[master_csv['Region Name']=='Greeley, CO']
csprings_df = master_csv.loc[master_csv['Region Name']=='Colorado Springs, CO']
breck_df = master_csv.loc[master_csv['Region Name']=='Breckenridge, CO; CO']
breck_df['Indicator ID'].value_counts()

Z3BR    225
ZSFH    225
Z5BR    225
ZCON    225
ZATT    225
Z1BR    225
Z2BR    225
ZABT    225
Z4BR    225
ZALL    225
Name: Indicator ID, dtype: int64

In [62]:
#ISAM & IRSW for Denver
denver_isam = denver_df.loc[denver_df['Indicator ID']=='ISAM']

denver_iram = denver_df.loc[denver_df['Indicator ID']=='IRAM']
#denver_iram
denver_isam

,Year,Month,Year-Month,Date Zillow,Date Interest,Indicator ID,Indicator Name,Region ID,Region Type,Region Name,Value,County Population,Unemployment Rate,Wells Fargo Rate,Freddie Mac Rate
10280,2017,Oct,2017-10,10/31/2017,10/5/2017,ISAM,"For-Sale Inventory (Smooth, All Homes, Monthly)",394530,metro,"Denver, CO",11047.0,706615,2.9,4.00,3.85
10281,2017,Oct,2017-10,10/31/2017,10/12/2017,ISAM,"For-Sale Inventory (Smooth, All Homes, Monthly)",394530,metro,"Denver, CO",11047.0,706615,2.9,4.00,3.91
10282,2017,Oct,2017-10,10/31/2017,10/19/2017,ISAM,"For-Sale Inventory (Smooth, All Homes, Monthly)",394530,metro,"Denver, CO",11047.0,706615,2.9,4.00,3.88
10283,2017,Oct,2017-10,10/31/2017,10/26/2017,ISAM,"For-Sale Inventory (Smooth, All Homes, Monthly)",394530,metro,"Denver, CO",11047.0,706615,2.9,4.13,3.94
10588,2017,Nov,2017-11,11/30/2017,11/2/2017,ISAM,"For-Sale Inventory (Smooth, All Homes, Monthly)",394530,metro,"Denver, CO",9969.0,706615,2.9,4.00,3.94
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62298,2020,Aug,2020-8,8/31/2020,8/6/2020,ISAM,"For-Sale Inventory (Smooth, All Homes, Monthly)",394530,metro,"Denver, CO",11276.0,738598,6.7,3.00,2.88
62299,2020,Aug,2020-8,8/31/2020,8/13/2020,ISAM,"For-Sale Inventory (Smooth, All Homes, Monthly)",394530,metro,"Denver, CO",11276.0,738598,6.7,3.00,2.96
62300,2020,Aug,2020-8,8/31/2020,8/20/2020,ISAM,"For-Sale Inventory (Smooth, All Homes, Monthly)",394530,metro,"Denver, CO",11276.0,738598,6.7,2.88,2.99
62301,2020,Aug,2020-8,8/31/2020,8/27/2020,ISAM,"For-Sale Inventory (Smooth, All Homes, Monthly)",394530,metro,"Denver, CO",11276.0,738598,6.7,2.88,2.91


In [25]:
#ISAM & IRSW for Colorado Springs
greeley_isam = greeley_df.loc[greeley_df['Indicator ID']=='ISAM']
#greeley_isam

In [22]:
#ISAM & ISRW for Greeley
csprings_isam = csprings_df.loc[csprings_df['Indicator ID']=='ISAM']
#csprings_isam

In [23]:
#ISAM & ISRW for Breckenridge
breck_isam = breck_df.loc[breck_df['Indicator ID']=='ISAM']
breck_isam

,Year,Month,Year-Month,Date Zillow,Date Interest,Indicator ID,Indicator Name,Region ID,Region Type,Region Name,Value,County Population,Unemployment Rate,Wells Fargo Rate,Freddie Mac Rate


In [4]:
indicators_url = "https://www.quandl.com/api/v3/datatables/ZILLOW/INDICATORS.json?&api_key="
query_url = indicators_url + quandl_key
response = requests.get(query_url).json()
#pprint(json.dumps(response, indent=4, sort_keys=True))

In [5]:
regions_url = "https://www.quandl.com/api/v3/datatables/ZILLOW/regions.json?region_type=city&api_key="
rquery_url = regions_url + quandl_key
r_response = requests.get(rquery_url).json()
#print(json.dumps(r_response, indent=4, sort_keys=True))
region_ids = pd.read_csv('data/zillow_region_ids.csv')
#region_ids
#region_ids[region_ids['Neighborhood'].str.contains('Denver')]
region_ids[region_ids['State_abrv'].str.contains('CO')]


,Region_id,Region_type,City_Name,State_abrv,Neighborhood,County
184,9594,city,Rocky Ford,CO,Otero County,NaN
206,9547,city,Weston,CO,Las Animas County,NaN
298,9328,city,Tabernash,CO,Grand County,NaN
395,9094,city,Pleasant View,CO,Montezuma County,NaN
405,9061,city,Mesa,CO,Grand Junction,Mesa County
...,...,...,...,...,...,...
9793,41283,city,Trinidad,CO,Las Animas County,NaN
9853,41152,city,Swink,CO,Otero County,NaN
9875,41110,city,Sugar City,CO,Crowley County,NaN
9958,4095,city,Chromo,CO,Archuleta County,NaN


In [33]:
co_region_ids = [11093, 16846, 5519, 20793, 34781, 10181, 44515, 38992, 33334, 10574, 23984, 17545, 19055, 17163, 4592,
                 26213, 119842, 50547, 45470, 24634, 119404,
                 12834, 24367, 120381, 7811, 38807, 46297, 17755, 52003, 4548, 119221, 13759, 119157, 55662, 4197,
                 16865, 120317, 37852, 36384, 7507, 44909, 119469, 8471, 39098, 52230, 38324, 120500, 13288, 16394,
                 49882, 120020, 395432, 21854, 17754, 119632, 22800, 119114, 56787, 24366, 24665, 4960, 52196, 120107,
                 8179, 119390, 36867, 16729, 46104, 46323, 24170, 120333, 11824, 50963, 24590, 11751, 31633, 12042,
                 120074, 13856, 45936, 55580, 39666, 54425, 396043, 42203, 23185, 30476, 6510, 396035, 16589, 50328]

In [42]:
#data = quandl.get_table('ZACKS/FC', paginate=True, ticker=['AAPL', 'MSFT'],
#per_end_date={'gte': '2015-01-01'}, qopts={'columns':['ticker', 'per_end_date']})
isaw_data = quandl.get_table("ZILLOW/DATA", api_key=quandl_key,
                             indicator_id='ISAW', paginate=True)#, region_id = co_region_ids)
isaw_data



#np.where(data['region_id']==4592)
#per_end_date={'gte': '2016-04-26'}

,indicator_id,region_id,date,value
None,,,,
0,ISAW,753924,2020-10-03,3541.0
1,ISAW,753924,2020-09-26,3560.0
2,ISAW,753924,2020-09-19,3587.0
3,ISAW,753924,2020-08-29,3774.0
4,ISAW,753924,2020-08-22,3771.0
...,...,...,...,...
17627,ISAW,102001,2017-11-04,1325207.0
17628,ISAW,102001,2017-10-28,1329289.0
17629,ISAW,102001,2017-10-21,1334286.0


In [49]:
data3.loc[data3['region_id']=='753924']

,region_id,region_type,region
None,,,
22766,753924,metro,"Urban Honolulu, HI"


In [47]:
data3

,region_id,region_type,region
None,,,
0,99999,zip,98847; WA; Wenatchee; Chelan County; Peshastin
1,99998,zip,98846; WA; Okanogan County; Pateros
2,99997,zip,98845; WA; Wenatchee; Douglas County; Palisades
3,99996,zip,98844; WA; Okanogan County; Oroville
4,99995,zip,98843; WA; Wenatchee; Douglas County; Orondo
...,...,...,...
78508,100000,zip,98848; WA; Moses Lake; Grant County; Quincy
78509,10000,city,Bloomington; MD; Garrett County
78510,1000,county,Echols County; GA; Valdosta


In [38]:
z1br= quandl.get_table('Zillow/Data', api_key=quandl_key, indicator_id = 'Z1BR', paginate=True, region_id = co_region_ids)
z1br

,indicator_id,region_id,date,value
None,,,,
0,Z1BR,8471,2020-09-30,723751.0
1,Z1BR,8471,2020-08-31,718174.0
2,Z1BR,8471,2020-07-31,713465.0
3,Z1BR,8471,2020-06-30,722672.0
4,Z1BR,8471,2020-05-31,724446.0
...,...,...,...,...
17001,Z1BR,10181,1996-05-31,83774.0
17002,Z1BR,10181,1996-04-30,83497.0
17003,Z1BR,10181,1996-03-31,83065.0


In [59]:
isaw = quandl.get_table('Zillow/Data', api_key=quandl_key, indicator_id = 'ISAW', paginate=True, region_id = co_metro_ids)
#isaw
isaw["date"]= pd.to_datetime(isaw["date"])
isaw_dated =isaw.loc[isaw["date"] > "2016-04-01"]
#isaw
isaw_dated

,indicator_id,region_id,date,value
None,,,,
0,ISAW,395007,2020-10-03,510.0
1,ISAW,395007,2020-09-26,518.0
2,ISAW,395007,2020-09-19,520.0
3,ISAW,395007,2020-08-29,545.0
4,ISAW,395007,2020-08-22,551.0
...,...,...,...,...
919,ISAW,394405,2017-11-04,1152.0
920,ISAW,394405,2017-10-28,1174.0
921,ISAW,394405,2017-10-21,1186.0


In [84]:
#quandl.bulkdownload("ZILLOW/DATA", quandl_key)
#zillow_data = quandl.export_table('ZILLOW/DATA', api_key=quandl_key)

In [3]:
data2 = quandl.get_table("ZILLOW/INDICATORS", api_key="YOUR KEY HERE")
data2

,indicator_id,indicator,category
None,,,
0,ZSFH,ZHVI Single-Family Homes Time Series ($),Home values
1,ZCON,ZHVI Condo/Co-op Time Series ($),Home values
2,ZATT,ZHVI All Homes- Top Tier Time Series ($),Home values
3,ZALL,"ZHVI All Homes (SFR, Condo/Co-op) Time Series ($)",Home values
4,ZABT,ZHVI All Homes- Bottom Tier Time Series ($),Home values
5,Z5BR,ZHVI 5+ Bedroom Time Series ($),Home values
6,Z4BR,ZHVI 4-Bedroom Time Series ($),Home values
7,Z3BR,ZHVI 3-Bedroom Time Series ($),Home values
8,Z2BR,ZHVI 2-Bedroom Time Series ($),Home values


In [42]:
#REGIONS TABLE
data3 = quandl.get_table("ZILLOW/REGIONS", api_key=quandl_key, paginate=True)
data3
#FOUND CO COUNTIES
co_region = data3[data3['region'].str.contains('CO')]
co_county = co_region[co_region['region_type']=='county']
co_county_ids = co_county['region_id'].tolist()
co_metros = co_region[co_region['region_type']=='metro']
co_metro_ids = co_metros['region_id'].tolist()
print(co_metro_ids)
#print(co_county_ids)

['753921', '753881', '753874', '753871', '395130', '395007', '394876', '394645', '394639', '394606', '394602', '394556', '394547', '394530', '394484', '394437', '394405']


In [49]:

#co_regions = quandl.get_table('ZILLOW/REGIONS', api_key=quandl_key, paginate = True)#, region_id=co_region_ids)
co_regions = data3.loc[data3['region_type']=='metro']
co_regions = co_regions.loc[co_regions['region'].str.contains('CO')]
co_regions

,region_id,region_type,region
None,,,
22769,753921,metro,"Steamboat Springs, CO; CO"
22805,753881,metro,"Glenwood Springs, CO; CO"
22812,753874,metro,"Craig, CO; CO"
22814,753871,metro,"Breckenridge, CO; CO"
50367,395130,metro,"Sterling, CO; CO"
50484,395007,metro,"Pueblo, CO; CO"
50612,394876,metro,"Montrose, CO; CO"
50833,394645,metro,"Greeley, CO"
50839,394639,metro,"Grand Junction, CO; CO"


In [57]:
breck_data = quandl.get_table("ZILLOW/DATA", api_key = quandl_key, paginate = True, region_id=394530)
breck_data['indicator_id'].value_counts()

SSSW    654
SASW    654
SSAW    654
SAAW    654
SRAW    654
SRSW    654
ZALL    297
ZSFH    297
ZCON    297
Z1BR    297
ZATT    297
Z4BR    297
Z3BR    297
ZABT    297
Z2BR    297
Z5BR    297
IRAW    154
ISAW    154
ISSW    154
IRSW    154
SAAM    151
SASM    151
SSSM    151
SSAM    151
SRAM    151
SRSM    151
CSAW    142
CSSW    142
MSAW    141
NRAW    141
MRAW    141
NSAW    141
CRSW    129
CRAW    129
LRAW    115
LRSW    115
LSAW    112
LSSW    112
RSSA     81
RSNA     81
IRAM     36
ISAM     36
IRSM     36
ISSM     36
MSAM     33
CSAM     33
NSAM     33
CSSM     33
MRAM     33
NRAM     33
CRAM     30
CRSM     30
LRSM     27
LRAM     27
LSSM     23
LSAM     23
Name: indicator_id, dtype: int64